## Setup

In [1]:
import numpy as np
from tensorflow.keras.optimizers import Adam
import tf_agents

2022-03-03 19:02:12.624286: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-03 19:02:12.624339: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.



In [2]:
from tf_agents.agents import DdpgAgent
from tf_agents.agents import ddpg

from tf_agents.environments import py_environment
from tf_agents.environments import tf_environment
from tf_agents.environments import tf_py_environment
from tf_agents.environments import utils
from tf_agents.specs import array_spec
from tf_agents.environments import wrappers
from tf_agents.environments import suite_gym
from tf_agents.trajectories import time_step as ts
from tf_agents.metrics import py_metrics
from tf_agents.drivers import py_driver,dynamic_episode_driver,dynamic_step_driver
from tf_agents.policies import random_py_policy,random_tf_policy,PyTFEagerPolicy
from tf_agents.replay_buffers import tf_uniform_replay_buffer,py_uniform_replay_buffer

In [5]:
from __future__ import absolute_import, division, print_function

import base64
# import imageio
import IPython
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image
# import pyvirtualdisplay

import tensorflow as tf

from tf_agents.agents.dqn import dqn_agent
from tf_agents.drivers import py_driver
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.networks import sequential
from tf_agents.policies import py_tf_eager_policy
from tf_agents.policies import random_tf_policy
from tf_agents.replay_buffers import reverb_replay_buffer
from tf_agents.replay_buffers import reverb_utils
from tf_agents.trajectories import trajectory
from tf_agents.specs import tensor_spec
from tf_agents.utils import common

## Hyperparameters

In [6]:
# from Maxime
nb_actions = 10
replay_buffer_capacity = 1000
log_interval=100
eval_interval=1000

# from DQN Tutorial
num_iterations = 20000 # @param {type:"integer"}

initial_collect_steps = 100  # @param {type:"integer"}
collect_steps_per_iteration = 1# @param {type:"integer"}
replay_buffer_max_length = 100000  # @param {type:"integer"}

batch_size = 64  # @param {type:"integer"}
learning_rate = 1e-3  # @param {type:"number"}
log_interval = 200  # @param {type:"integer"}

num_eval_episodes = 10  # @param {type:"integer"}
eval_interval = 1000  # @param {type:"integer"}

## Environement

In [7]:
# Input des quotas à produire

quota_laitue = 3
quota_carotte = 2
quota_broccoli = 0
quota_tomate = 1
quota_chou = 1
quota_haricot = 0
quota_patate = 4
quota_ail = 3
quota_oignon = 0
quota_courgette = 1

In [8]:
# Définir l'ensemble des tuiles name = [id, prix, rendement, quota]

ensemble_des_tuiles = dict()

ensemble_des_tuiles[1] = ['tomate',4,3,quota_tomate]
ensemble_des_tuiles[2] = ['carotte',2,5,quota_carotte]
ensemble_des_tuiles[3] = ['broccoli',6,1,quota_broccoli]
ensemble_des_tuiles[4] = ['chou',3,3,quota_chou]
ensemble_des_tuiles[5] = ['laitue',2,1,quota_laitue]
ensemble_des_tuiles[6] = ['haricot',6,2,quota_haricot]
ensemble_des_tuiles[7] = ['patate',2,4,quota_patate]
ensemble_des_tuiles[8] = ['ail',8,1,quota_ail]
ensemble_des_tuiles[9] = ['oignon',8,1,quota_oignon]
ensemble_des_tuiles[10] = ['courgette',1,2,quota_courgette]

nb_ensemble_des_tuiles = len(ensemble_des_tuiles)
print(nb_ensemble_des_tuiles)
ensemble_des_tuiles

10


{1: ['tomate', 4, 3, 1],
 2: ['carotte', 2, 5, 2],
 3: ['broccoli', 6, 1, 0],
 4: ['chou', 3, 3, 1],
 5: ['laitue', 2, 1, 3],
 6: ['haricot', 6, 2, 0],
 7: ['patate', 2, 4, 4],
 8: ['ail', 8, 1, 3],
 9: ['oignon', 8, 1, 0],
 10: ['courgette', 1, 2, 1]}

In [9]:
# Définir l'ensemble des rewards et des quotas

ensemble_des_rewards = []
ensemble_des_quotas = []

for k,v in ensemble_des_tuiles.items():
    ensemble_des_rewards.append(v[2])
    
for k,v in ensemble_des_tuiles.items():
    ensemble_des_quotas.append(v[-1])
    
ensemble_des_rewards, ensemble_des_quotas

([3, 5, 1, 3, 1, 2, 4, 1, 1, 2], [1, 2, 0, 1, 3, 0, 4, 3, 0, 1])

In [10]:
# Définir l'ensemble des actions possibles

import itertools

nb_tuiles = nb_ensemble_des_tuiles
dim_x = 25
dim_y = 1
ensemble_des_possibles = dict()

compteur = -1
for x,y,t in itertools.product(range(dim_x),range(dim_y),range(1, nb_tuiles+1)):
    compteur += 1
    ensemble_des_possibles[compteur] = [x,y,t]
    
nb_ensemble_des_possibles = len(ensemble_des_possibles)
print(nb_ensemble_des_possibles)
ensemble_des_possibles

250


{0: [0, 0, 1],
 1: [0, 0, 2],
 2: [0, 0, 3],
 3: [0, 0, 4],
 4: [0, 0, 5],
 5: [0, 0, 6],
 6: [0, 0, 7],
 7: [0, 0, 8],
 8: [0, 0, 9],
 9: [0, 0, 10],
 10: [1, 0, 1],
 11: [1, 0, 2],
 12: [1, 0, 3],
 13: [1, 0, 4],
 14: [1, 0, 5],
 15: [1, 0, 6],
 16: [1, 0, 7],
 17: [1, 0, 8],
 18: [1, 0, 9],
 19: [1, 0, 10],
 20: [2, 0, 1],
 21: [2, 0, 2],
 22: [2, 0, 3],
 23: [2, 0, 4],
 24: [2, 0, 5],
 25: [2, 0, 6],
 26: [2, 0, 7],
 27: [2, 0, 8],
 28: [2, 0, 9],
 29: [2, 0, 10],
 30: [3, 0, 1],
 31: [3, 0, 2],
 32: [3, 0, 3],
 33: [3, 0, 4],
 34: [3, 0, 5],
 35: [3, 0, 6],
 36: [3, 0, 7],
 37: [3, 0, 8],
 38: [3, 0, 9],
 39: [3, 0, 10],
 40: [4, 0, 1],
 41: [4, 0, 2],
 42: [4, 0, 3],
 43: [4, 0, 4],
 44: [4, 0, 5],
 45: [4, 0, 6],
 46: [4, 0, 7],
 47: [4, 0, 8],
 48: [4, 0, 9],
 49: [4, 0, 10],
 50: [5, 0, 1],
 51: [5, 0, 2],
 52: [5, 0, 3],
 53: [5, 0, 4],
 54: [5, 0, 5],
 55: [5, 0, 6],
 56: [5, 0, 7],
 57: [5, 0, 8],
 58: [5, 0, 9],
 59: [5, 0, 10],
 60: [6, 0, 1],
 61: [6, 0, 2],
 62: [6, 0, 

In [11]:
class IArchitectEnv(py_environment.PyEnvironment):
    def __init__(self,verbose=False):
        # Initialisation environment
        self._state = np.zeros((dim_x,),dtype=np.int32)
        self._action_spec = array_spec.BoundedArraySpec(
            shape=(), dtype=np.int32, minimum=0, maximum=nb_ensemble_des_possibles-1, name='action')
        self._observation_spec = array_spec.BoundedArraySpec(
            shape=(dim_x,), dtype=np.int32, minimum=0, maximum=nb_tuiles, name='observation')
        self._episode_ended = False
        self.verbose = verbose
        
        # Initialisation rewards/quotas
        self.REWARD = 1000
        self.PENALTY_ALREADY_FILLED = -100
        self.DEFAULT_REWARD = ensemble_des_rewards
        self.QUOTA = ensemble_des_quotas
        self.COUNTER = 0

        
    def action_spec(self):
        return self._action_spec

    
    def observation_spec(self):
        return self._observation_spec

    
    def _reset(self):
        self._state = np.zeros((dim_x,),dtype=np.int32)
        self._episode_ended = False
        self.COUNTER = 0
        return ts.restart(self._state)


    def _step(self, action):
        
        while self.COUNTER <= 20:
            
            x, y, t = ensemble_des_possibles[int(action)]
            
            unique, counts = np.unique(self._state, return_counts=True)
            quota_en_cours = dict(zip(unique, counts))
            quota_a_faire = self.QUOTA
            reward_policy = self.DEFAULT_REWARD
            quota_restant = []
        
            for i in quota_a_faire:
                quota_restant.append(quota_a_faire[i] - quota_en_cours.get(i,0))
        
        
            if self._episode_ended:
                return self.reset()

            if self.verbose:
                print(self._state,action,x,y,t)

            # Définition de la reward par défaut
            reward = 1

            # Analyse des conditions de défaite
            if self._state[x] > 0: # Contrôler la défaite
                reward = self.PENALTY_ALREADY_FILLED
            
            self._state[x] = t # Définition de l'action
            
            # Définition de la reward
            if sum(1 for number in quota_restant if number<=0)==len(quota_restant):
                reward = reward_policy[t-1]/2
            else:
                reward = reward_policy[t-1]*2
                
# Contrôler la victoire
        # Analyse des conditions de victoire
            if 0 not in self._state:
                self._episode_ended = True
                reward = self.REWARD
        
            if not self._episode_ended:
                result = ts.transition(self._state, reward=reward, discount=1.0)
            else:
                result = ts.termination(self._state, reward=reward)
        
            if self.verbose:
                print(result.observation,reward)
                
            self.COUNTER += 1
            
            return result

In [12]:
# Initialiser les environnement
environment = IArchitectEnv(verbose=True)
train_env = tf_py_environment.TFPyEnvironment(environment)
eval_env = tf_py_environment.TFPyEnvironment(environment)

utils.validate_py_environment(environment, episodes=5)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] 203 20 0 4
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 4 0 0 0 0] 6
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 4 0 0 0 0] 97 9 0 8
[0 0 0 0 0 0 0 0 0 8 0 0 0 0 0 0 0 0 0 0 4 0 0 0 0] 2
[0 0 0 0 0 0 0 0 0 8 0 0 0 0 0 0 0 0 0 0 4 0 0 0 0] 8 0 0 9
[9 0 0 0 0 0 0 0 0 8 0 0 0 0 0 0 0 0 0 0 4 0 0 0 0] 2
[9 0 0 0 0 0 0 0 0 8 0 0 0 0 0 0 0 0 0 0 4 0 0 0 0] 134 13 0 5
[9 0 0 0 0 0 0 0 0 8 0 0 0 5 0 0 0 0 0 0 4 0 0 0 0] 2
[9 0 0 0 0 0 0 0 0 8 0 0 0 5 0 0 0 0 0 0 4 0 0 0 0] 76 7 0 7
[9 0 0 0 0 0 0 7 0 8 0 0 0 5 0 0 0 0 0 0 4 0 0 0 0] 8
[9 0 0 0 0 0 0 7 0 8 0 0 0 5 0 0 0 0 0 0 4 0 0 0 0] 86 8 0 7
[9 0 0 0 0 0 0 7 7 8 0 0 0 5 0 0 0 0 0 0 4 0 0 0 0] 8
[9 0 0 0 0 0 0 7 7 8 0 0 0 5 0 0 0 0 0 0 4 0 0 0 0] 71 7 0 2
[9 0 0 0 0 0 0 2 7 8 0 0 0 5 0 0 0 0 0 0 4 0 0 0 0] 10
[9 0 0 0 0 0 0 2 7 8 0 0 0 5 0 0 0 0 0 0 4 0 0 0 0] 117 11 0 8
[9 0 0 0 0 0 0 2 7 8 0 8 0 5 0 0 0 0 0 0 4 0 0 0 0] 2
[9 0 0 0 0 0 0 2 7 8 0 8 0 5 0 0 0 0 0 0 4 0 0 0 0] 22 2 0 3
[9 0 3 0 0 0 

AttributeError: 'NoneType' object has no attribute 'is_last'

## Agent

In [ ]:
fc_layer_params = (200, 100)
action_tensor_spec = tensor_spec.from_spec(environment.action_spec())
num_actions = action_tensor_spec.maximum - action_tensor_spec.minimum + 1

# Define a helper function to create Dense layers configured with the right
# activation and kernel initializer.
def dense_layer(num_units):
      return tf.keras.layers.Dense(
          num_units,
          activation=tf.keras.activations.relu,
          kernel_initializer=tf.keras.initializers.VarianceScaling(
              scale=2.0, mode='fan_in', distribution='truncated_normal'))

# QNetwork consists of a sequence of Dense layers followed by a dense layer
# with `num_actions` units to generate one q_value per available action as
# its output.
dense_layers = [dense_layer(num_units) for num_units in fc_layer_params]
q_values_layer = tf.keras.layers.Dense(
    num_actions,
    activation=None,
    kernel_initializer=tf.keras.initializers.RandomUniform(
        minval=-0.03, maxval=0.03),
    bias_initializer=tf.keras.initializers.Constant(-0.2))
q_net = sequential.Sequential(dense_layers + [q_values_layer])

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

train_step_counter = tf.Variable(0)

agent = dqn_agent.DqnAgent(
    train_env.time_step_spec(),
    train_env.action_spec(),
    q_network=q_net,
    optimizer=optimizer,
    td_errors_loss_fn=common.element_wise_squared_loss,
    train_step_counter=train_step_counter)

agent.initialize()

In [ ]:
q_net.summary()

## Policies

In [ ]:
eval_policy = agent.policy
collect_policy = agent.collect_policy

In [ ]:
random_policy = random_tf_policy.RandomTFPolicy(train_env.time_step_spec(),
                                                train_env.action_spec())

## Metrics

In [ ]:
def compute_avg_return(environment, policy, num_episodes=10):

    total_return = 0.0
    for _ in range(num_episodes):

        time_step = environment.reset()
        episode_return = 0.0

        while not time_step.is_last():
            action_step = policy.action(time_step)
            time_step = environment.step(action_step.action)
            episode_return += time_step.reward
        total_return += episode_return

    avg_return = total_return / num_episodes
    return avg_return.numpy()[0]


# See also the metrics module for standard implementations of different metrics.
# https://github.com/tensorflow/agents/tree/master/tf_agents/metrics

In [ ]:
avg_return = compute_avg_return(eval_env, random_policy, num_eval_episodes)
returns = [avg_return]

## Replay Buffer

In [ ]:
replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
                                data_spec=agent.collect_data_spec,                                                     
                                batch_size=train_env.batch_size,                                                             
                                max_length=100000)

def collect_step(environment, policy, buffer):
    time_step = environment.current_time_step()
    action_step = policy.action(time_step)
    next_time_step = environment.step(action_step.action)
    traj = trajectory.from_transition(time_step, 
                                      action_step, 
                                      next_time_step)
# Add trajectory to the replay buffer
    buffer.add_batch(traj)

In [ ]:
train_env.batch_size

In [ ]:
collect_steps_per_iteration = 1
batch_size = 64
dataset = replay_buffer.as_dataset(num_parallel_calls=3, 
                                   sample_batch_size=batch_size, 
                                   num_steps=2).prefetch(3)
iterator = iter(dataset)

In [ ]:
result_loss = []
num_iterations = 20000
train_env.reset()
for _ in range(batch_size):
    collect_step(train_env, agent.policy, replay_buffer)
for _ in range(num_iterations):
    # Collect a few steps using collect_policy and save to the replay buffer.
    for _ in range(collect_steps_per_iteration):
        collect_step(train_env, agent.collect_policy, replay_buffer)
    # Sample a batch of data from the buffer and update the agent's network.
    experience, unused_info = next(iterator)
    train_loss = agent.train(experience).loss
    step = agent.train_step_counter.numpy()
    # Print loss every 200 steps.
    if step % 200 == 0:
        print('step = {0}: loss = {1}'.format(step, train_loss))
        result_loss.append(train_loss)
    # Evaluate agent's performance every 1000 steps.
    if step % 1000 == 0:
        avg_return = compute_avg_return(train_env, agent.policy, 5)
        print('step = {0}: Average Return = {1}'.format(step, avg_return))
        returns.append(avg_return)

In [ ]:
plt.plot(result_loss)
plt.ylabel('Loss')
plt.xlabel('Iterations')

In [ ]:
plt.plot(returns)
plt.ylabel('Average Return')
plt.xlabel('Iterations')

## Evaluation

In [ ]:
observers = []
driver = py_driver.PyDriver(
    eval_env, PyTFEagerPolicy(agent.policy), observers, max_steps=1000, max_episodes=10)

In [ ]:
time_spec_init = eval_env.reset()
driver.run(time_spec_init)